In [9]:
cascadeClassifier = cv2.CascadeClassifier("path/to/your/cascade.xml")

# Check if the classifier was loaded successfully
if cascadeClassifier.empty():
    print("Error loading cascade classifier.")
else:
    print("Cascade classifier loaded successfully.")

Error loading cascade classifier.


In [10]:
import cv2

cam = cv2.VideoCapture(0)

face_cascade_file = 'haarcascade_frontalface_default.xml'
face_detector = cv2.CascadeClassifier(face_cascade_file)

if face_detector.empty():
    print("Error: Unable to load the face cascade classifier.")
    print(f"Make sure '{face_cascade_file}' is present in the current directory.")
    exit()

face_id = input('\n Make sure the first user entered is 0. Enter user id and press <return> ==>  ')

print("\n [INFO] Initializing face capture. Look at the camera and wait ...")
count = 0

while True:
    ret, img = cam.read()
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        count += 1

        # Save the captured image into the datasets folder
        face_image = gray[y:y + h, x:x + w]
        cv2.imwrite(f"dataset/User.{face_id}.{count}.jpg", face_image)

    cv2.imshow('image', img)

    k = cv2.waitKey(100) & 0xff
    if k == 27:
        break
    elif count >= 70:
        break

print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 Make sure the first user entered is 0. Enter user id and press <return> ==>  0

 [INFO] Initializing face capture. Look at the camera and wait ...

 [INFO] Exiting Program and cleanup stuff


In [20]:
import os
import numpy as np
from PIL import Image
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.layers import Dense, Activation, Flatten
from keras.utils import to_categorical
from keras import callbacks
from sklearn.model_selection import train_test_split

# Path for face image database
path = 'dataset'

def downsample_image(img):
    img = Image.fromarray(img.astype('uint8'), 'L')
    img = img.resize((32, 32), Image.ANTIALIAS)
    return np.array(img)

# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    faceSamples = []
    ids = []

    for imagePath in imagePaths:
        # if there is an error saving any jpegs
        try:
            PIL_img = Image.open(imagePath).convert('L')  # convert it to grayscale
        except:
            continue
        img_numpy = np.array(PIL_img, 'uint8')

        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faceSamples.append(img_numpy)
        ids.append(id)

    # Convert to NumPy array with dtype=object to handle variable-length arrays
    faceSamples = np.array([downsample_image(ab) for ab in faceSamples], dtype=object)

    return faceSamples, ids

print("\n [INFO] Training faces now.")
faces, ids = getImagesAndLabels(path)

# Set up the model
n_faces = len(set(ids))
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_faces, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.save('trained_model.h5')

# Prepare the data
faces = np.asarray(faces)
faces = np.array([downsample_image(ab) for ab in faces])
ids = np.asarray(ids)
faces = faces[:, :, :, np.newaxis]
print("Shape of Data: " + str(faces.shape))
print("Number of unique faces : " + str(n_faces))

ids = to_categorical(ids)

faces = faces.astype('float32')
faces /= 255.

x_train, x_test, y_train, y_test = train_test_split(faces, ids, test_size=0.2, random_state=0)

checkpoint = callbacks.ModelCheckpoint('trained_model.h5', monitor='val_accuracy',
                                       save_best_only=True, save_weights_only=True, verbose=1)

model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test),
          shuffle=True, callbacks=[checkpoint])

# Print the number of faces trained and end the program
print("\n [INFO] " + str(n_faces) + " faces trained. Exiting Program")


 [INFO] Training faces now.
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 15, 15, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_43 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 6, 6, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_12 (Flatten)        (None, 2304)              0         
                                                                 
 dense_24 (Dense)       

C:\Users\Asif_\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to trained_model.h5
2/2 [==============================] - 1s 165ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 47ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 49ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4: val_accuracy did not 

In [ ]:
import numpy as np
import cv2
import os
from PIL import Image
import numpy as np
from keras.models import load_model

# Load the pre-trained model
face_model = load_model('trained_model.keras')

cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
font = cv2.FONT_HERSHEY_SIMPLEX

def start():
    cap = cv2.VideoCapture(0)
    print('here')
    ret = True

    clip = []
    while ret:
        ret, frame = cap.read()
        nframe = frame
        faces = faceCascade.detectMultiScale(
            frame,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )

        try:
            (x,y,w,h) = faces[0]
        except:
            continue
        frame = frame[y:y+h,x:x+w]
        frame = cv2.resize(frame, (32, 32))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('result small' , frame)
        c= cv2.waitKey(1)
        if c & 0xFF == ord('q'):
            break
        
        gray = gray.reshape(-1, 32, 32, 1).astype('float32') / 255.
        print(gray.shape)
        prediction = face_model.predict(gray)
        print("prediction:" + str(prediction))

        print("\n\n\n\n")
        print("----------------------------------------------")
        prediction = prediction.tolist()
        
        listv = prediction[0]
        n = listv.index(max(listv))
        print("\n")
        print("----------------------------------------------")
        print( "Highest Probability: " + "User " + str(n) + "==>" + str(prediction[0][n]) )
        print("----------------------------------------------")
        print("\n")
        for (x, y, w, h) in faces:
            try:
                cv2.rectangle(nframe, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(nframe, str(labels[n]), (x+5,y-5), font, 1, (255,255,255), 2)
            except:
                la = 2 
        prediction = np.argmax(face_model.predict(gray), 1)
        print(prediction)
        cv2.imshow('result', nframe)
        c = cv2.waitKey(1)
        if c & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

start()

here
(1, 32, 32, 1)
1/1 [==============================] - 0s 161ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 35ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 20ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 20ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 22ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 19ms/step
[0]
(1, 32, 32, 1)
1/1 

prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 20ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 17ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 18ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 19ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 19ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 20ms/step
prediction:[[1.]]





--

1/1 [==============================] - 0s 21ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 28ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 25ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 19ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 21ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 19ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [====================

1/1 [==============================] - 0s 22ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 20ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 20ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 20ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 22ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 20ms/step
[0]
(1, 32, 32, 1)
1/1 [====================

1/1 [==============================] - 0s 19ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 19ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 18ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 19ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 19ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 18ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [====================

1/1 [==============================] - 0s 21ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 20ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 21ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 20ms/step
[0]
(1, 32, 32, 1)
1/1 [==============================] - 0s 18ms/step
prediction:[[1.]]





----------------------------------------------


----------------------------------------------
Highest Probability: User 0==>1.0
----------------------------------------------


1/1 [==============================] - 0s 19ms/step
[0]
(1, 32, 32, 1)
1/1 [====================